In [1]:
import pandas as pd

In [127]:
sites = pd.read_excel('data/raw_data/los_angeles/Appendix H MasterListbasic.xlsx')

sites = sites.dropna(axis='rows', how='all').dropna(axis='columns', how='all')
sites['APN'] = sites['APN'].astype(str)

In [219]:
# Downloaded from https://data.lacity.org/Community-Economic-development/Building-Permits-New-Housing-Units/cpkv-aajs
permits = pd.read_csv('data/raw_data/los_angeles/Building_Permits__New_Housing_Units.csv')

permits['Issue Date'] = pd.to_datetime(permits['Issue Date'])

/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [220]:
permits_with_apn = permits[
    permits['Assessor Book'].notnull()
    & ~permits['Assessor Page'].str.contains('\*').fillna(False)
    & ~permits['Assessor Page'].str.contains('BRK').fillna(False)
    & ~permits['Assessor Parcel'].str.contains('\*').fillna(False)
    & ~permits['Assessor Parcel'].str.contains('XXX').fillna(False)
].copy()

In [221]:
permits_with_apn['Assessor Book'] = permits_with_apn['Assessor Book'].astype(int).astype(str)
permits_with_apn['Assessor Page'] = permits_with_apn['Assessor Page'].astype(int).astype(str)
permits_with_apn['Assessor Parcel'] = permits_with_apn['Assessor Parcel'].astype(int).astype(str)

In [222]:
len(permits) - len(permits_with_apn)

736

In [223]:
permits_with_apn['APN'] = (
    permits_with_apn['Assessor Book'].astype(int).astype(str)
    + permits_with_apn['Assessor Page'].astype(int).astype(str).str.zfill(3)
    + permits_with_apn['Assessor Parcel'].astype(str).str.zfill(3)
)

In [206]:
permits_with_apn['Permit Type'].value_counts()

Bldg-New             17930
Bldg-Alter/Repair     1284
Bldg-Addition          887
Fire Sprinkler           7
Bldg-Relocation          5
Nonbldg-New              5
Plumbing                 4
Grading                  2
HVAC                     1
Elevator                 1
Electrical               1
Bldg-Demolition          1
Name: Permit Type, dtype: int64

In [208]:
permits_with_apn[
    (permits_with_apn['Permit Type'] == 'Bldg-New')
]['Permit Sub-Type'].value_counts()

1 or 2 Family Dwelling    16005
Apartment                  1695
Commercial                  230
Name: Permit Sub-Type, dtype: int64

In [225]:
new_housing_permits = permits_with_apn[
    (permits_with_apn['Permit Type'] == 'Bldg-New')
    & (permits_with_apn['Permit Sub-Type'].isin(['Apartment', '1 or 2 Family Dwelling']))
]

In [226]:
new_housing_permits.shape

(17700, 53)

In [227]:
new_housing_permits['Issue Date'].max()

Timestamp('2021-03-12 00:00:00')

In small lot subdivisions and duplexes, often separate buildings in the same project have separate rows in the permits. Let's dedupe them by just keeping the last row.

In [358]:
new_housing_permits_deduped = new_housing_permits.sort_values(['APN', 'Issue Date']).drop_duplicates('APN', keep='last')

The fifth SCAG cycle covers Oct 2013 to Oct 2021. Let's just look at 2014 - 2020 (7 years) for simplicity. We'll multiply this number by 8/7 at the end to account for the fact that the cycle is actually 8 years long.

We'll also try looking at just 2014-2019, excluding 2020, to account for COVID weirdness. That's 6 years, so multiply by 8/6 to get the 8-year average.

In [359]:
new_housing_permits_2014_to_2020 = new_housing_permits_deduped[
    (new_housing_permits_deduped['Issue Date'] >= '2014-01-01')
    & (new_housing_permits_deduped['Issue Date'] < '2021-01-01')
]

In [360]:
new_housing_permits_2014_to_2019 = new_housing_permits_deduped[
    (new_housing_permits_deduped['Issue Date'] >= '2014-01-01')
    & (new_housing_permits_deduped['Issue Date'] < '2020-01-01')
]

In [191]:
# Had to look up some zones at https://codelibrary.amlegal.com/codes/los_angeles/latest/lapz/
zone_mappings = {
    'R1': 'R',
    'R2': 'R',
    'R3': 'R',
    'R4': 'R',
    'R5': 'R',
    'RD1.5': 'R',
    'RD2': 'R',
    'RD3': 'R',
    'RD5': 'R',
    'RD6': 'R',
    'RE1': 'R',  # residential estate
    'RE2': 'R',  # residential estate
    'RE4': 'R',  # residential estate
    'RE9': 'R',  # residential estate
    'RA-': 'R',  # residential/accessory services - mixed use
    'RS-': 'R',  # "Suburban Zone" lol
    'RW2': 'R',  # Residential Waterways
    'RAS3': 'C',  # mixed use
    'C1': 'C',
    'C2': 'C',
    'C4': 'C',
    'C5': 'C',
    'C5': 'C',
    'CM': 'M',  # commercial manufacturing
    'CR': 'C',  # not sure
    'P-1': 'C',  # not sure
    'WC': 'C',  # allows retail according to https://planning.lacity.org/ordinances/docs/CommercialCannabis/files/SummaryLocationRestrictions.pdf
    'M1': 'M',
    'PF': 'Public/Other',  # public facilities
    'OS': 'Public/Other',  # open space
}

In [192]:
zones = sites['Zone'].drop_duplicates()

not_found_zones = []
for zone in zones:
    if not any(key in zone for key in zone_mappings):
        not_found_zones.append(zone)

In [193]:
print('\n'.join(sorted(not_found_zones)))

In [196]:
def get_zone_mapping(zone):
    for zone_match_string, zone_type in zone_mappings.items():
        if zone_match_string in zone:
            return zone_type
    return None

sites_with_type = sites.copy()
sites_with_type['Zone Type'] = sites_with_type['Zone'].map(get_zone_mapping)

In [198]:
sites_with_type['Zone Type'].value_counts()

C               11003
R                9675
M                 631
Public/Other       21
Name: Zone Type, dtype: int64

In [428]:
new_housing_permits_2014_to_2020['APN'].isin(
    sites_with_type['APN']
).mean()

0.0746379502413665

In [435]:
new_housing_permits_2014_to_2020[
    new_housing_permits_2014_to_2020['APN'].isin(
        sites_with_type['APN']
    )
]['# of Residential Dwelling Units'].sum()

21266

In [436]:
new_housing_permits_2014_to_2020['# of Residential Dwelling Units'].sum()

60534

In [437]:
21266 / 60534

0.3513067036706644

In [428]:
new_housing_permits_2014_to_2020['APN'].isin(
    sites_with_type['APN']
).mean()

0.0746379502413665

## Merge the datasets

Let's look at through 2020:

In [361]:
merged_through_2020 = sites_with_type.merge(
    new_housing_permits_2014_to_2020,
    on='APN',
    how='left',
    indicator=True
)

In [362]:
merged_through_2020['_merge'].value_counts(normalize=True)

left_only     0.962307
both          0.037693
right_only    0.000000
Name: _merge, dtype: float64

In [363]:
merged_through_2019 = sites_with_type.merge(
    new_housing_permits_2014_to_2019,
    on='APN',
    how='left',
    indicator=True
)

In [364]:
merged_through_2019['_merge'].value_counts(normalize=True)

left_only     0.967135
both          0.032865
right_only    0.000000
Name: _merge, dtype: float64

## Development probabilities by zone

In [367]:
development_probability_through_2020 = merged_through_2020.groupby('Zone Type').apply(lambda g: (g['_merge'] == 'both').mean())

In [368]:
development_probability_through_2020

Zone Type
C               0.011179
M               0.006339
Public/Other    0.000000
R               0.069974
dtype: float64

In [369]:
8/7 * development_probability_through_2020

Zone Type
C               0.012776
M               0.007245
Public/Other    0.000000
R               0.079970
dtype: float64

And now through 2019 only:

In [427]:
merged_through_2019['_merge'].value_counts(normalize=True)

left_only     0.967135
both          0.032865
right_only    0.000000
Name: _merge, dtype: float64

In [438]:
8/6 * 0.032865

0.04382

In [425]:
development_probability_through_2019 = merged_through_2019.groupby('Zone Type').apply(lambda g: (g['_merge'] == 'both').mean())

In [371]:
development_probability_through_2019

Zone Type
C               0.010270
M               0.004754
Public/Other    0.000000
R               0.060465
dtype: float64

In [372]:
8/6 * development_probability_through_2019

Zone Type
C               0.013693
M               0.006339
Public/Other    0.000000
R               0.080620
dtype: float64

Let's use the 2019 numbers, because they're more optimistic.

The residential development fraction seems pretty high! But actually it's probably lower when you exclude buildings that were just developed for single-fam with no net new units.

In [417]:
merged_through_2020.groupby('Zone Type').apply(lambda g: (g['_merge'] == 'both').sum())

Zone Type
C               123
M                 4
Public/Other      0
R               677
dtype: int64

In [418]:
merged_through_2019.groupby('Zone Type').apply(lambda g: (g['_merge'] == 'both').sum())

Zone Type
C               113
M                 3
Public/Other      0
R               585
dtype: int64

In [421]:
merged_through_2020[
    merged_through_2020['Zone Type'] == 'R'
]['Permit Sub-Type'].value_counts()

Apartment                 385
1 or 2 Family Dwelling    292
Name: Permit Sub-Type, dtype: int64

In [422]:
residential_zone_one_or_two_units = merged_through_2020[
    (merged_through_2020['_merge'] == 'both')
    & (merged_through_2020['Zone Type'] == 'R')
    & (merged_through_2020['Permit Sub-Type'] == '1 or 2 Family Dwelling')
]

Let's see how many of these are SFDs...

In [400]:
residential_zone_one_or_two_units[['Work Description']].sample(12)

,Work Description
20870,"NEW 3-STORY, TYPE-VB DUPLEX (REAR)"
5954,A NEW THREE STORY DUPLEX WITH A CARPOT
2584,"FOR NEW THREE STORY SFD/GAR PER VTT 72367-5L-IA (LOT 1), Comply with"
11807,New 2 story Duplex building #2
2934,"NEW DUPLEX/GAR- BLD-B, 2 OF 2"
16733,SUPPLEMENTAL PERMIT TO REVISE WORK DESCRIPTION FROM ADDITION TO - NEW
11171,(N) 2 STORY DUPLEX WITH ATTACHED 3 CAR GARAGE
18193,NEW 4 STORY SFD IN A SMALL LOT SUBDIVISION. BUILDING TYPE A. SEE COM
20887,NEW 3-STORY DUPLEX WITH ATTACHED 2 CAR GARAGE @ FRONT LOT
18529,(N) 2 STORY DUPLEX/ ROOF DECK W/ BASEMENT 4-CAR GARAGE AND STORAGE RO


Some of these might be small lot subdivisions...

In [404]:
residential_zone_one_or_two_units['Work Description'][
    residential_zone_one_or_two_units['Work Description'].str.lower().str.contains('small lot')
]

1698      NEW SINGLE FAMILY DWELLING W/ATTACHED 2 CAR GARAGE; PART OF SMALL LOT
2608                                   SMALL LOT SUDDIVISION. Lot 3 Plan Type C
2617                         CONSTRUCTION OF NEW 4 UNITS SMALL LOT SFR.  3 OF 4
2698                                       3 STORY SMALL LOT SUBDIVISION UNIT B
2987                           5 lot small lot subdivision. Lot 3- plan type A2
2990     NEW 4 UNIT SMALL LOT SUBDIVISION  (4 STORY TYPE VB SFD WITH ATTACHED G
3084     New 3 story unit, part of 8 unit small lot subdivision, Unit B3.  (28.
3129     (N) 4-story SFD with attached garage; Type VB (PLAN 3) Small lot subdi
3453     Small lot subdivision pursuant to Tentative Tract Map # VTT-73277-SL f
3902     NEW SFD/GARAGE - LOT 1, PLAN-1. -  1 OF 6. " Small lot subdivision pur
4354     CONSTRUCTION OF A  SFD   PER  4-UNIT SMALL LOT SUBDIVISION  AA-2014-26
4367      NEW SFD/ GAR " Small lot subdivision pursuant to Tentative Tract  Map
6143      SMALL LOT SUBDIVISION PURSUANT

In [405]:
residential_zone_one_or_two_units['Work Description'][
    residential_zone_one_or_two_units['Work Description'].str.lower().str.contains('small lot')
].shape

(29,)

Okay, only 29 of them. Those should count as new multifamily. Let's see how many are non-small lot subdivision SFDs...

In [416]:
residential_zone_one_or_two_units.shape

(292, 63)

In [413]:
sfhs = residential_zone_one_or_two_units[
    ~residential_zone_one_or_two_units['Work Description'].str.lower().str.contains('small lot')
    & ~residential_zone_one_or_two_units['Work Description'].str.lower().str.contains('duplex')
]

In [414]:
sfhs['Work Description'].sample(20)

16550     NEW  3 STORY TWO FAMILY DWELLING  WITH ATTACHED 2- CAR GARAGE AND ROO
7176     NEW 2-STORY  55' 2" X 33' IRREGULAR-SHAPED SINGLE FAMILY DWELLING WITH
4420                             NEW 2 STORY SFD W/BASEMENT AND ATTACHED GARAGE
7247                                             NEW SFD W/ATT GARAGE - PLAN 2A
13232    SUPPLEMENTAL TO PERMIT 19010-70000-04313 TO REVISE BUILDING HEIGHT FRO
2312     New 103'-1''x 109'-8'' 2-story single family dwelling with attached ga
14908    Lot 4: plan-A2 to construct a new 1708 sf , three story, 45'0" tall, s
5576     lot 13, 3-story s.f.d. with attached 2-car garage as part of a 14-unit
2584      FOR NEW THREE STORY SFD/GAR PER VTT 72367-5L-IA  (LOT 1), Comply with
21133    NEW 19'9"  X 40'0"  FOUR STORY DWELLING WITH AN ATTACHED GARAGE (LOT D
749                   NEW TWO STORY SINGLE FAMILY DWELLING W/ATT.  4 CAR GARAGE
17337    (VTT 72661-SL) THREE STORY SFD W/ ATT GARAGE & OPEN ROOF DECK (PLAN B)
19135    Proposed three story single fam

In [415]:
sfhs.shape

(85, 63)

So probably 85 out of 677, or roughly 12.5% of the residential-zone development only produced a non-SLS single family house, and therefore likely added no net units.

It's possible that some of the duplexes are also no net units, but it's not easy to figure that out because the "Current Use" data seems so low-quality.

## Development probabilities by current use

In [269]:
pd.set_option('max_rows', 300)

In [ ]:
current_use_types = {
    'Commercial': 'Commercial',
    'Store': 'Commercial',
    'VACANT LAND': 'Vacant',
    'Double, duplex, or two units': 'Residential',
    
}

In [277]:
sites_with_type[
    (sites_with_type['Zone Type'] == 'R')
    & (sites_with_type['Current Use'] == 'Commercial')
]

,APN,Year Built,Current Use,Zone,30 du/ac (Y/N),Net Units,Acres,CPA,General Plan Land Use Designation,Zone Type
12,2535014026,1940.0,Commercial,RD1.5-1-O,No,6.0,0.244444,Arleta - Pacoima,Low Medium II Residential,R
45,2620025009,1923.0,Commercial,R1-1-O,No,5.0,0.700161,Arleta - Pacoima,Low Residential,R
46,2620025026,1925.0,Commercial,R1-1-O,No,5.0,0.751561,Arleta - Pacoima,Low Residential,R
48,2621010002,1947.0,Commercial,(T)R1-1,No,6.0,0.828719,Arleta - Pacoima,Low Residential,R
51,2622004020,1946.0,Commercial,(T)RD2-1,No,6.0,0.337351,Arleta - Pacoima,Low Medium II Residential,R
...,...,...,...,...,...,...,...,...,...,...
21322,5539028002,1904.0,Commercial,R3-1,YES,8.0,0.172153,Wilshire,Medium Residential,R
21323,5539028003,0.0,Commercial,R3-1,YES,8.0,0.172153,Wilshire,Medium Residential,R
21324,5539028004,1910.0,Commercial,R3-1,YES,8.0,0.172176,Wilshire,Medium Residential,R
21327,5539028013,1910.0,Commercial,R3-1,YES,8.0,0.172153,Wilshire,Medium Residential,R


In [280]:
merged_through_2019[
    (merged_through_2019['Zone Type'] == 'R')
    & (merged_through_2019['_merge'] == 'both')
]

,APN,Year Built,Current Use,Zone_x,30 du/ac (Y/N),Net Units,Acres,CPA,General Plan Land Use Designation,Zone Type,...,Applicant Business Name,Applicant Address 1,Applicant Address 2,Applicant Address 3,Zone_y,Occupancy,Floor Area-L.A. Building Code Definition,Census Tract,Latitude/Longitude,_merge
101,4357016029,0.0,Single Residence,RE15-1-H,No,6.0,2.200938,Bel Air - Beverly Crest,Very Low II Residential,R,...,NaN,23622 CALABASAS RD,NaN,"CALABASAS, CA",RE15-1-H,NaN,36183.0,2612.0,"(34.09373, -118.43372)",both
249,5175007005,1901.0,Commercial,R3-1,YES,7.0,0.150666,Boyle Heights,Medium Residential,R,...,NaN,1005 E. 80TH ST,NaN,LA CA,R3-1,NaN,729.6,2036.0,"(34.0505, -118.20583)",both
250,5175007005,1901.0,Commercial,R3-1,YES,7.0,0.150666,Boyle Heights,Medium Residential,R,...,NaN,1005 E. 80TH ST,NaN,LA CA,R3-1,NaN,729.6,2036.0,"(34.05058, -118.20578)",both
294,5175033005,1905.0,Commercial,R3-1,YES,7.0,0.149219,Boyle Heights,Medium Residential,R,...,NaN,NaN,NaN,NaN,R3-1-CUGU,NaN,3910.0,2036.0,NaN,both
295,5175033005,1905.0,Commercial,R3-1,YES,7.0,0.149219,Boyle Heights,Medium Residential,R,...,NaN,NaN,NaN,NaN,R3-1-CUGU,NaN,3669.0,2036.0,"(34.05107, -118.21318)",both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21910,5522031006,1913.0,Commercial,R3-1,YES,8.0,0.166575,Wilshire,Medium Residential,R,...,NaN,NaN,NaN,NaN,R3-1,NaN,2388.0,1924.2,"(34.07741, -118.3104)",both
21911,5522031006,1913.0,Commercial,R3-1,YES,8.0,0.166575,Wilshire,Medium Residential,R,...,NaN,NaN,NaN,NaN,R3-1,NaN,2388.0,1924.2,"(34.07741, -118.3103)",both
21935,5522033017,1920.0,"Double, duplex, or two units",R3-1,YES,7.0,0.166437,Wilshire,Medium Residential,R,...,NaN,NaN,NaN,NaN,R3-1,NaN,14522.0,1924.2,"(34.07711, -118.31194)",both
21940,5522033025,1918.0,"Double, duplex, or two units",R3-1,YES,6.0,0.164463,Wilshire,Medium Residential,R,...,NaN,NaN,NaN,NaN,R3-1,NaN,14000.0,1924.2,"(34.0768, -118.31337)",both


In [285]:
merged_through_2019[
    (merged_through_2019['Zone Type'] == 'R')
    & (merged_through_2019['_merge'] == 'both')
]['Permit Sub-Type'].value_counts()

1 or 2 Family Dwelling    917
Apartment                 384
Name: Permit Sub-Type, dtype: int64

In [287]:
merged_through_2019[
    (merged_through_2019['Zone Type'] == 'R')
    & (merged_through_2019['_merge'] == 'both')
]['# of Residential Dwelling Units'].value_counts()

1.0      643
2.0      274
3.0       32
8.0       30
6.0       26
5.0       23
9.0       17
4.0       16
12.0      15
7.0       14
15.0      13
13.0      11
10.0       9
19.0       8
18.0       8
11.0       7
24.0       7
32.0       7
23.0       6
16.0       6
14.0       5
28.0       5
22.0       5
20.0       5
29.0       5
21.0       5
46.0       4
37.0       4
17.0       4
41.0       4
49.0       4
44.0       4
27.0       4
26.0       4
31.0       3
34.0       3
25.0       3
33.0       3
93.0       2
84.0       2
65.0       2
50.0       2
35.0       2
60.0       2
47.0       2
51.0       2
61.0       2
36.0       2
68.0       2
101.0      2
89.0       1
144.0      1
30.0       1
136.0      1
75.0       1
76.0       1
120.0      1
88.0       1
273.0      1
86.0       1
74.0       1
309.0      1
83.0       1
70.0       1
666.0      1
85.0       1
80.0       1
48.0       1
69.0       1
81.0       1
199.0      1
90.0       1
45.0       1
224.0      1
39.0       1
99.0       1
703.0      1

In [290]:
merged_through_2019[
    (merged_through_2019['# of Residential Dwelling Units'] == 1)
    & (merged_through_2019['_merge'] == 'both')
]['Current Use'].value_counts()

VACANT LAND                          200
Commercial                           185
Double, duplex, or two units          63
Churches                              55
Single Residence                      54
Schools (Private)                     25
Studios                               21
Auto Service Shops                    12
Homes for the aged and others         12
Three units (any combination)         11
3 units (4 stories or less)           10
Service Station                        8
2 units (4 stories or less)            6
4 units (4 stories or less)            6
Medical/dental Building                4
Office Building                        1
Five or more apartments or units       1
Store and residential combination      1
Name: Current Use, dtype: int64

In [291]:
merged_through_2019[
    (merged_through_2019['# of Residential Dwelling Units'] == 2)
    & (merged_through_2019['_merge'] == 'both')
]['Current Use'].value_counts()

Commercial                              216
Double, duplex, or two units             35
VACANT LAND                              22
Store and residential combination        10
Single Residence                          8
Auto Service Shops                        5
Five or more apartments or units          4
Miscellaneous commercial                  4
Store                                     4
Restaurants, cocktail lounge, tavern      4
Four units (any combination)              3
Medical/dental Building                   2
Churches                                  2
Lots - patron or employee                 2
Store and office combination              1
Three units (any combination)             1
Name: Current Use, dtype: int64

In [286]:
merged_through_2019[
    (merged_through_2019['Zone Type'] == 'R')
    & (merged_through_2019['_merge'] == 'both')
]['Net Units'].value_counts()

7.0      212
6.0      204
8.0      183
5.0      170
10.0      88
13.8      55
9.0       47
27.0      38
14.0      30
11.0      28
16.0      24
19.0      21
85.0      21
12.0      20
18.0      20
37.0      20
13.0      17
29.7      14
15.0      14
17.0      11
64.0      10
24.0       8
30.0       7
28.0       4
26.0       4
22.0       2
42.0       2
21.0       2
32.0       2
20.0       2
190.0      2
95.0       1
142.0      1
36.0       1
23.0       1
34.0       1
31.0       1
200.0      1
40.0       1
146.0      1
38.3       1
84.0       1
53.0       1
60.0       1
29.0       1
1.2        1
35.0       1
41.0       1
145.0      1
54.0       1
Name: Net Units, dtype: int64

In [278]:
sites_with_type[
    (sites_with_type['Zone Type'] == 'R')
    & (sites_with_type['Current Use'] == 'Commercial')
]['CPA'].value_counts()

Wilshire                            901
Southeast Los Angeles               468
Westlake                            295
West Adams - Baldwin Hills          287
Hollywood                           257
South Los Angeles                   231
Palms - Mar Vista - Del Rey         230
North Hollywood - Valley Village    216
Silver Lake - Echo Park             115
Northeast Los Angeles               100
Boyle Heights                        85
Sherman Oaks - Studio City           62
Central City North                   56
West Los Angeles                     49
Van Nuys - North Sherman Oaks        45
Harbor Gateway                       35
Sun Valley - La Tuna Canyon          34
Mission Hills - Panorama City        21
Westchester - Playa del Rey          20
Canoga Park - Woodland Hills         19
Sunland - Tujunga                    16
Sylmar                               15
Venice                               14
Arleta - Pacoima                     12
San Pedro                            11


In [276]:
for zone_type in set(zone_mappings.values()):
    rows = sites_with_type[
        sites_with_type['Zone Type'] == zone_type
    ]
    print(zone_type)
    print(rows['Current Use'].value_counts())
    print()
# merged_through_2020['Current Use'].value_counts()

R
Commercial                                         3626
Double, duplex, or two units                       1385
VACANT LAND                                         797
Single Residence                                    458
Lots - patron or employee                           385
Churches                                            360
Four units (any combination)                        342
Three units (any combination)                       286
2 units (4 stories or less)                         228
Five or more apartments or units                    188
Government Owned Property (OPEN)                    147
Schools (Private)                                   136
5 or more units (4 stories or less)                 101
Parking Lot (Patron or Employee)                    100
4 units (4 stories or less)                          78
Office Building                                      71
3 units (4 stories or less)                          68
Store                                         

# Validation

Okay, these look okay (though the C fraction is depressingly low). Let's do some sanity checks:

In [243]:
merged_through_2019[
    merged_through_2019['_merge'] == 'both'
]['Zone Type'].value_counts()

R    1499
C     179
M       4
Name: Zone Type, dtype: int64

In [245]:
merged_through_2019[
    merged_through_2019['_merge'] == 'both'
]['Permit Sub-Type'].value_counts()

1 or 2 Family Dwelling    1154
Apartment                  528
Name: Permit Sub-Type, dtype: int64

In [248]:
sites_with_type['Zone Type'].value_counts()

C               11003
R                9675
M                 631
Public/Other       21
Name: Zone Type, dtype: int64

In [249]:
sites_with_type[
    sites_with_type['Zone Type'] == 'R'
]

,APN,Year Built,Current Use,Zone,30 du/ac (Y/N),Net Units,Acres,CPA,General Plan Land Use Designation,Zone Type
0,2524020030,1955.0,Churches,R1-1,No,6.0,1.032943,Arleta - Pacoima,Low Residential,R
4,2527005023,1951.0,Churches,R1-1,No,6.0,0.929683,Arleta - Pacoima,Low Residential,R
5,2527008039,1957.0,Churches,R1-1,No,8.0,0.929040,Arleta - Pacoima,Low Residential,R
9,2533009902,0.0,Schools (Private),R1-1,No,6.0,0.688705,Arleta - Pacoima,Low Residential,R
10,2533009911,0.0,Schools (Private),R1-1,No,8.0,0.965970,Arleta - Pacoima,Low Residential,R
...,...,...,...,...,...,...,...,...,...,...
21325,5539028006,1927.0,"Double, duplex, or two units",R3-1,YES,9.0,0.172153,Wilshire,Medium Residential,R
21326,5539028011,1930.0,Light manufacturing,R3-1,YES,9.0,0.177365,Wilshire,Medium Residential,R
21327,5539028013,1910.0,Commercial,R3-1,YES,8.0,0.172153,Wilshire,Medium Residential,R
21328,5539028014,1910.0,Commercial,R3-1,YES,8.0,0.172176,Wilshire,Medium Residential,R


In [250]:
sites_with_type[
    sites_with_type['Zone Type'] == 'R'
]['Current Use'].value_counts()

Commercial                               3626
Double, duplex, or two units             1385
VACANT LAND                               797
Single Residence                          458
Lots - patron or employee                 385
                                         ... 
Tennis courts, clubs, Pro shops             1
Postal Facility                             1
Amusement Facility                          1
3 units (4 stories or less with Pool)       1
Single Residence with Therapy Pool          1
Name: Current Use, Length: 135, dtype: int64

In [252]:
sites_with_type[
    (sites_with_type['Zone Type'] == 'R')
    & (sites_with_type['Current Use'] == 'Commercial')
]['Zone'].value_counts()

R3-1               2148
R4-1                337
R3-1-O              179
R3-1VL              124
RD1.5-1              96
R3-1XL               95
R1-1                 95
R4-1VL               92
RD1.5-1XL            54
R4-2                 52
R4(CW)-75/3-O        46
R4(CW)-75/3          41
[Q]R3-1              40
R4-1L                17
R2-1XL               13
R2-1                 12
RD2-1                12
R4(CW)-75/3.7-O      12
R1-1-RFA             11
R5(CW)-U/6           11
R2-1VL               11
R3-1-CDO              9
RD1.5-1-O             8
RE15-1-H              8
RD2-1XL               8
R3-2                  7
RD1.5-1VL             7
R1-1VL                6
(T)RS-1               6
[Q]R4-1VL             6
RS-1                  5
(T)R1-1               5
[T]RD2-1              4
R1-1-O                4
(T)R3-1               4
RD2-1VL               3
(F)R2-1               3
R4-2-O                3
RD2-1-O               3
R4-1L-O               3
RD1.5-1-CDO           3
R1-1XL          